# from suite2p import run_s2p
import json
from pathlib import Path
import numpy as np
import tables
import shutil
# import flammkuchen as fl
from tqdm import tqdm
import json
import hdf5plugin
from shutil import copy


In [5]:
master_path =  Path(r"Z:\Anja & Hagar")
files = list(master_path.glob("*_v*"))

fast_path_dest =  Path(r"C:\Users\lavian\temp_suite2p")

unprocessed = [f for f in files if not (f / "suite2p" / "combined").exists()]
unprocessed

[WindowsPath('Z:/Anja & Hagar/230906_f0_ipn_v10'),
 WindowsPath('Z:/Anja & Hagar/230907_f0_post_v10'),
 WindowsPath('Z:/Anja & Hagar/230907_f1_aHB_v10'),
 WindowsPath('Z:/Anja & Hagar/230908_f2_post_v10')]

In [3]:

def restructure_for_suite2p(path, ds_name="original"):
    ds_path = path / ds_name

    suite_2p_dir = (path / "suite2p")
    suite_2p_dir.mkdir(exist_ok=True)
    for f in tqdm(list(ds_path.glob("*.h5"))):
        dest_dir = suite_2p_dir / f.stem
        dest_dir.mkdir(exist_ok=True)
        copy(f,dest_dir / f.name)

In [6]:
for i in unprocessed[-1:]:
    restructure_for_suite2p(i)

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [12:29<00:00, 29.99s/it]


In [ ]:
unprocessed[3]

In [ ]:
data_path = unprocessed[5]
print(data_path)
restructure_for_suite2p(data_path)

In [8]:
for i in unprocessed[-1:]:
    data_path = i
    for dataset_path in tqdm((data_path / "suite2p").glob("[0-9][0-9][0-9][0-9]")):
        print("#####################################")

        fast_path = fast_path_dest
        fast_path.mkdir(exist_ok=True)
        print(data_path)
        try:
            with open(next((data_path / "behavior").glob("*metadata.json")), "r") as f:
                metadata = json.load(f)
            fs = metadata["imaging"]["microscope_config"]["scanning"]["framerate"]  # sampling frequency
        except:
            try:
                with open(next((data_path / "behavior").glob("*/*metadata.json")), "r") as f:
                    metadata = json.load(f)
                fs = metadata["imaging"]["microscope_config"]["scanning"]["framerate"]  # sampling frequency
            except:
                fs = 3
                    
        ops = {
            # file paths
            'look_one_level_down': False, # whether to look in all subfolders when searching for tiffs
            'fast_disk': str(fast_path), # used to store temporary binary file, defaults to save_path0
            'delete_bin': True, # whether to delete binary file after processing
            'mesoscan': False, # for reading in scanimage mesoscope files
            'bruker': False, # whether or not single page BRUKER tiffs!
            'h5py': str(dataset_path), # take h5py as input (deactivates data_path)
            'h5py_key': 'stack_4D', #key in h5py where data array is stored
            'save_path0': str(dataset_path), # stores results, defaults to first item in data_path
            'save_folder': str(dataset_path),
            'subfolders': [],
            'move_bin': False, # if 1, and fast_disk is different than save_disk, binary file is moved to save_disk

            # main settings
            'nplanes' : 1, # each tiff has these many planes in sequence
            'nchannels' : 1, # each tiff has these many channels per plane
            'functional_chan' : 1, # this channel is used to extract functional ROIs (1-based)
            'tau':  1., # this is the main parameter for deconvolution
            'fs': fs,  # sampling rate (PER PLANE e.g. for 12 plane recordings it will be around 2.5)
            'force_sktiff': False, # whether or not to use scikit-image for tiff reading
            'frames_include': -1,
            "num_workers": 10,

            # output settings
            'preclassify': 0., # apply classifier before signal extraction with probability 0.3
            'save_mat': False, # whether to save output as matlab files
            'combined': True, # combine multiple planes into a single result /single canvas for GUI
            'aspect': 1.0, # um/pixels in X / um/pixels in Y (for correct aspect ratio in GUI)

            # bidirectional phase offset
            'do_bidiphase': False,
            'bidiphase': 0,
            'bidi_corrected': False,

            # registration settings
            'do_registration': 1, # whether to register data (2 forces re-registration)
            'two_step_registration': False,
            'keep_movie_raw': False,
            'nimg_init': 300, # subsampled frames for finding reference image
            'batch_size': 500, # number of frames per batch
            'maxregshift': 0.1, # max allowed registration shift, as a fraction of frame max(width and height)
            'align_by_chan' : 1, # when multi-channel, you can align by non-functional channel (1-based)
            'reg_tif': False, # whether to save registered tiffs
            'reg_tif_chan2': False, # whether to save channel 2 registered tiffs
            'subpixel' : 5, # precision of subpixel registration (1/subpixel steps)
            'smooth_sigma_time' : 0, # gaussian smoothing in time
            'smooth_sigma': 1.15, # ~1 good for 2P recordings, recommend >5 for 1P recordings
            'th_badframes': 1.0, # this parameter determines which frames to exclude when determining cropping - set it smaller to exclude more frames
            'pad_fft': False,

            # non rigid registration settings
            'nonrigid': False, # whether to use nonrigid registration
            'block_size': [128, 128], # block size to register (** keep this a multiple of 2 **)
            'snr_thresh': 1.2, # if any nonrigid block is below this threshold, it gets smoothed until above this threshold. 1.0 results in no smoothing
            'maxregshiftNR': 5, # maximum pixel shift allowed for nonrigid, relative to rigid

             # 1P settings
            '1Preg': False, # whether to perform high-pass filtering and tapering
            'spatial_hp': 40, # window for spatial high-pass filtering before registration
            'pre_smooth': 2, # whether to smooth before high-pass filtering before registration
            'spatial_taper': 50, # how much to ignore on edges (important for vignetted windows, for FFT padding do not set BELOW 3*ops['smooth_sigma'])

            # cell detection settings
            'roidetect': True, # whether or not to run ROI extraction
            'sparse_mode': False, # whether or not to run sparse_mode
            'diameter': 6, # if not sparse_mode, use diameter for filtering and extracting
            'spatial_scale': 4, # 0: multi-scale; 1: 6 pixels, 2: 12 pixels, 3: 24 pixels, 4: 48 pixels
            'connected': 0, # whether or not to keep ROIs fully connected (set to 0 for dendrites)
            'nbinned': 5000, # max number of binned frames for cell detection
            'max_iterations': 30, # maximum number of iterations to do cell detection
            'threshold_scaling': 1.6
            , # adjust the automatically determined threshold by this scalar multiplier
            'max_overlap': 0.9, # cells with more overlap than this get removed during triage, before refinement
            'high_pass': 100, # running mean subtraction with window of size 'high_pass' (use low values for 1P)

            # ROI extraction parameters
            'inner_neuropil_radius': 2, # number of pixels to keep between ROI and neuropil donut
            'min_neuropil_pixels': 350, # minimum number of pixels in the neuropil
            'allow_overlap': False, # pixels that are overlapping are thrown out (False) or added to both ROIs (True)

            # channel 2 detection settings (stat[n]['chan2'], stat[n]['not_chan2'])
            'chan2_thres': 0.65, # minimum for detection of brightness on channel 2

            # deconvolution settings
            'spikedetect': False, # whether or not to run spike deconvolution
            'baseline': 'maximin', # baselining mode (can also choose 'prctile')
            'win_baseline': 60., # window for maximin
            'sig_baseline': 10., # smoothing constant for gaussian filter
            'prctile_baseline': 8.,# optional (whether to use a percentile baseline)
            'neucoeff': .7,  # neuropil coefficient
            'xrange': np.array([0, 0]),
            'yrange': np.array([0, 0]),
        }

        db = {'data_path': [str(dataset_path)]}

        opsEnd=run_s2p(ops=ops, db=db)
        #except (UnboundLocalError, ValueError):
       #     print("Error for fish ", path)

        shutil.rmtree(fast_path)

0it [00:00, ?it/s]

#####################################
Z:\Anja & Hagar\230908_f2_post_v10
{'data_path': ['Z:\\Anja & Hagar\\230908_f2_post_v10\\suite2p\\0000']}
h5
** Found 1 h5 files - converting to binary **
NOTE: using a list of h5 files:
['Z:\\Anja & Hagar\\230908_f2_post_v10\\suite2p\\0000\\0000.h5']
time 13.27 sec. Wrote 731 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 731 frames
Reference frame, 8.96 sec.
added enhanced mean image
----------- Total 16.92 sec
NOTE: applying default C:\Users\lavian\.suite2p\classifiers\classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 03
Binned movie [166,350,342] in 0.60 sec.
ROIs: 200, cost: 0.5536, time: 9.4191
ROIs: 320, cost: 0.5348, time: 12.2385
ROIs: 338, cost: 0.5318, time: 14.0699
ROIs: 338, cost: 0.5797, time: 15.1719
ROIs: 338, cost: 0.553

1it [00:54, 54.84s/it]

#####################################
Z:\Anja & Hagar\230908_f2_post_v10
{'data_path': ['Z:\\Anja & Hagar\\230908_f2_post_v10\\suite2p\\0001']}
h5
** Found 1 h5 files - converting to binary **
NOTE: using a list of h5 files:
['Z:\\Anja & Hagar\\230908_f2_post_v10\\suite2p\\0001\\0001.h5']
time 13.40 sec. Wrote 731 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 731 frames
Reference frame, 8.85 sec.
added enhanced mean image
----------- Total 17.02 sec
NOTE: applying default C:\Users\lavian\.suite2p\classifiers\classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 03
Binned movie [166,352,350] in 0.61 sec.
ROIs: 200, cost: 0.5598, time: 10.7212
ROIs: 323, cost: 0.5370, time: 14.1830
ROIs: 358, cost: 0.5312, time: 16.4012
ROIs: 365, cost: 0.5298, time: 18.2614
ROIs: 365, cost: 0.58

2it [01:55, 58.00s/it]

...

23it [20:50, 51.85s/it]

#####################################
Z:\Anja & Hagar\230908_f2_post_v10
{'data_path': ['Z:\\Anja & Hagar\\230908_f2_post_v10\\suite2p\\0023']}
h5
** Found 1 h5 files - converting to binary **
NOTE: using a list of h5 files:
['Z:\\Anja & Hagar\\230908_f2_post_v10\\suite2p\\0023\\0023.h5']
time 14.14 sec. Wrote 731 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 731 frames
Reference frame, 8.69 sec.
added enhanced mean image
----------- Total 16.77 sec
NOTE: applying default C:\Users\lavian\.suite2p\classifiers\classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 03
Binned movie [166,354,348] in 0.62 sec.
ROIs: 200, cost: 0.5793, time: 4.0329
ROIs: 400, cost: 0.5611, time: 6.5328
ROIs: 491, cost: 0.5546, time: 8.4327
ROIs: 499, cost: 0.5538, time: 9.9827
ROIs: 499, cost: 0.5691, 

24it [21:41, 51.77s/it]

#####################################
Z:\Anja & Hagar\230908_f2_post_v10
{'data_path': ['Z:\\Anja & Hagar\\230908_f2_post_v10\\suite2p\\0024']}
h5
** Found 1 h5 files - converting to binary **
NOTE: using a list of h5 files:
['Z:\\Anja & Hagar\\230908_f2_post_v10\\suite2p\\0024\\0024.h5']
time 13.88 sec. Wrote 731 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 731 frames
Reference frame, 8.75 sec.
added enhanced mean image
----------- Total 16.92 sec
NOTE: applying default C:\Users\lavian\.suite2p\classifiers\classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 03
Binned movie [166,354,346] in 0.62 sec.
ROIs: 200, cost: 0.5956, time: 4.2996
ROIs: 400, cost: 0.5752, time: 6.9163
ROIs: 486, cost: 0.5682, time: 9.0162
ROIs: 493, cost: 0.5671, time: 10.6161
ROIs: 493, cost: 0.5821,

25it [22:34, 54.17s/it]


In [ ]:

data_path = unprocessed[3]
dataset_path = data_path / 'suite2p' / '0002'
print("#####################################")

fast_path = fast_path_dest
fast_path.mkdir(exist_ok=True)
print(data_path)
try:
    with open(next((data_path / "behavior").glob("*metadata.json")), "r") as f:
        metadata = json.load(f)
    fs = metadata["imaging"]["microscope_config"]["scanning"]["framerate"]  # sampling frequency
except:
    try:
        with open(next((data_path / "behavior").glob("*/*metadata.json")), "r") as f:
            metadata = json.load(f)
        fs = metadata["imaging"]["microscope_config"]["scanning"]["framerate"]  # sampling frequency
    except:
        fs = 3

ops = {
    # file paths
    'look_one_level_down': False, # whether to look in all subfolders when searching for tiffs
    'fast_disk': str(fast_path), # used to store temporary binary file, defaults to save_path0
    'delete_bin': True, # whether to delete binary file after processing
    'mesoscan': False, # for reading in scanimage mesoscope files
    'bruker': False, # whether or not single page BRUKER tiffs!
    'h5py': str(dataset_path), # take h5py as input (deactivates data_path)
    'h5py_key': 'stack_4D', #key in h5py where data array is stored
    'save_path0': str(dataset_path), # stores results, defaults to first item in data_path
    'save_folder': str(dataset_path),
    'subfolders': [],
    'move_bin': False, # if 1, and fast_disk is different than save_disk, binary file is moved to save_disk

    # main settings
    'nplanes' : 1, # each tiff has these many planes in sequence
    'nchannels' : 1, # each tiff has these many channels per plane
    'functional_chan' : 1, # this channel is used to extract functional ROIs (1-based)
    'tau':  1., # this is the main parameter for deconvolution
    'fs': fs,  # sampling rate (PER PLANE e.g. for 12 plane recordings it will be around 2.5)
    'force_sktiff': False, # whether or not to use scikit-image for tiff reading
    'frames_include': -1,
    "num_workers": 10,

    # output settings
    'preclassify': 0., # apply classifier before signal extraction with probability 0.3
    'save_mat': False, # whether to save output as matlab files
    'combined': True, # combine multiple planes into a single result /single canvas for GUI
    'aspect': 1.0, # um/pixels in X / um/pixels in Y (for correct aspect ratio in GUI)

    # bidirectional phase offset
    'do_bidiphase': False,
    'bidiphase': 0,
    'bidi_corrected': False,

    # registration settings
    'do_registration': 1, # whether to register data (2 forces re-registration)
    'two_step_registration': False,
    'keep_movie_raw': False,
    'nimg_init': 300, # subsampled frames for finding reference image
    'batch_size': 500, # number of frames per batch
    'maxregshift': 0.1, # max allowed registration shift, as a fraction of frame max(width and height)
    'align_by_chan' : 1, # when multi-channel, you can align by non-functional channel (1-based)
    'reg_tif': False, # whether to save registered tiffs
    'reg_tif_chan2': False, # whether to save channel 2 registered tiffs
    'subpixel' : 5, # precision of subpixel registration (1/subpixel steps)
    'smooth_sigma_time' : 0, # gaussian smoothing in time
    'smooth_sigma': 1.15, # ~1 good for 2P recordings, recommend >5 for 1P recordings
    'th_badframes': 1.0, # this parameter determines which frames to exclude when determining cropping - set it smaller to exclude more frames
    'pad_fft': False,

    # non rigid registration settings
    'nonrigid': False, # whether to use nonrigid registration
    'block_size': [128, 128], # block size to register (** keep this a multiple of 2 **)
    'snr_thresh': 1.2, # if any nonrigid block is below this threshold, it gets smoothed until above this threshold. 1.0 results in no smoothing
    'maxregshiftNR': 5, # maximum pixel shift allowed for nonrigid, relative to rigid

     # 1P settings
    '1Preg': False, # whether to perform high-pass filtering and tapering
    'spatial_hp': 25, # window for spatial high-pass filtering before registration
    'pre_smooth': 2, # whether to smooth before high-pass filtering before registration
    'spatial_taper': 50, # how much to ignore on edges (important for vignetted windows, for FFT padding do not set BELOW 3*ops['smooth_sigma'])

    # cell detection settings
    'roidetect': True, # whether or not to run ROI extraction
    'sparse_mode': False, # whether or not to run sparse_mode
    'diameter': 6, # if not sparse_mode, use diameter for filtering and extracting
    'spatial_scale': 1, # 0: multi-scale; 1: 6 pixels, 2: 12 pixels, 3: 24 pixels, 4: 48 pixels
    'connected': 0, # whether or not to keep ROIs fully connected (set to 0 for dendrites)
    'nbinned': 5000, # max number of binned frames for cell detection
    'max_iterations': 20, # maximum number of iterations to do cell detection
    'threshold_scaling': 1.5
    , # adjust the automatically determined threshold by this scalar multiplier
    'max_overlap': 0.9, # cells with more overlap than this get removed during triage, before refinement
    'high_pass': 100, # running mean subtraction with window of size 'high_pass' (use low values for 1P)

    # ROI extraction parameters
    'inner_neuropil_radius': 2, # number of pixels to keep between ROI and neuropil donut
    'min_neuropil_pixels': 350, # minimum number of pixels in the neuropil
    'allow_overlap': False, # pixels that are overlapping are thrown out (False) or added to both ROIs (True)

    # channel 2 detection settings (stat[n]['chan2'], stat[n]['not_chan2'])
    'chan2_thres': 0.65, # minimum for detection of brightness on channel 2

    # deconvolution settings
    'spikedetect': False, # whether or not to run spike deconvolution
    'baseline': 'maximin', # baselining mode (can also choose 'prctile')
    'win_baseline': 60., # window for maximin
    'sig_baseline': 10., # smoothing constant for gaussian filter
    'prctile_baseline': 8.,# optional (whether to use a percentile baseline)
    'neucoeff': .7,  # neuropil coefficient
    'xrange': np.array([0, 0]),
    'yrange': np.array([0, 0]),
}

db = {'data_path': [str(dataset_path)]}

opsEnd=run_s2p(ops=ops, db=db)
#except (UnboundLocalError, ValueError):
#     print("Error for fish ", path)

shutil.rmtree(fast_path)